In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rmisra/news-headlines-dataset-for-sarcasm-detection")

print("Path to dataset files:", path)

100%|█| 3.30M/3.30M [00:01<00:00, 3.15MB/s]

Extracting files...
Path to dataset files: /home/zanchet/.cache/kagglehub/datasets/rmisra/news-headlines-dataset-for-sarcasm-detection/versions/2


In [11]:
import json

sentences = []
labels = []
urls = []
with open("Sarcasm_Headlines_Dataset.json", 'r') as f:
    for line in f:
        data = json.loads(line)
        sentences.append(data['headline'])
        labels.append(data['is_sarcastic'])
        urls.append(data['article_link'])

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [14]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)

In [28]:
word_index = tokenizer.word_index
vocab_size = len(word_index)

In [17]:
sequences = tokenizer.texts_to_sequences(sentences)

In [18]:
padded = pad_sequences(sequences, padding='post')

In [19]:
padded[0]

array([  308, 15115,   679,  3337,  2298,    48,   382,  2576, 15116,
           6,  2577,  8434,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0], dtype=int32)

In [20]:
padded.shape

(26709, 40)

In [22]:
validation_percentage = 15

ts = int(padded.shape[0] * (100 - validation_percentage) / 100)

X_train = padded[0:ts]
X_test = padded[ts:]

y_train = labels[0:ts]
y_test = labels[ts:]

In [23]:
X_train.shape

(22702, 40)

In [24]:
X_test.shape

(4007, 40)

In [25]:
import tensorflow as tf


In [30]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 600, input_length = X_train.shape[1]),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

/home/zanchet/dev/LAMIA/LAMIA_venv/lib64/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2024-11-19 11:35:28.691912: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
